# Setup Database for the Sales Assistant Agent
We are going to setup all the tools that will be used by the Agent. First we install some packages, get a session handler and create the database we are going to be using


In [ ]:
# Import python packages
import pandas as pd

# Get an active session for Snowpark:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
# Adding this in case you are not using ACCOUNTADMIN from a trial accout but
# another role with enough permissions to run this notebook

this_session_role = session.get_current_role()
print (this_session_role)

We are going to create a new database for this lab that will containt the data used by the Agents

In [ ]:
CREATE OR REPLACE DATABASE CC_SNOWFLAKE_INTELLIGENCE_E2E;
USE DATABASE CC_SNOWFLAKE_INTELLIGENCE_E2E;

create or replace stage docs ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true );
create or replace stage csv ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true );

Copy the files from the Git repository:

In [ ]:
session.file.put("csv/*", "@CSV", auto_compress = False)
session.file.put("docs/*", "@DOCS", auto_compress = False)

A wareshouse called COMPUTE_WH should be already created in any Trial account. Creating it just in case it does not exists.

In [ ]:
-- CREATE THE WH IF DOES NOT EXISTS
CREATE WAREHOUSE IF NOT EXISTS COMPUTE_WH
WAREHOUSE_SIZE = 'XSMALL';

In [ ]:
db = session.get_current_database()
print (db)

### Setup ROLES

We are going to have three different roles. One that can get access to only BIKE info, other to SNOW info and the third one that can access all

In [ ]:
create role if not exists BIKE_ROLE;
create role if not exists SNOW_ROLE;
create role if not exists BIKE_SNOW_ROLE;

GRANT ROLE BIKE_ROLE TO ROLE ACCOUNTADMIN;
GRANT ROLE SNOW_ROLE TO ROLE ACCOUNTADMIN;
GRANT ROLE BIKE_SNOW_ROLE TO ROLE ACCOUNTADMIN;

CREATE USER IF NOT EXISTS bike_user PASSWORD = 'Password123!' DEFAULT_ROLE = BIKE_ROLE;
grant role BIKE_ROLE to user bike_user;

CREATE USER IF NOT EXISTS snow_user PASSWORD = 'Password123!' DEFAULT_ROLE = SNOW_ROLE;
grant role SNOW_ROLE to user snow_user;

CREATE USER IF NOT EXISTS all_user PASSWORD = 'Password123!' DEFAULT_ROLE = BIKE_SNOW_ROLE;
grant role BIKE_SNOW_ROLE to user all_user;

In [ ]:
current_user = session.get_current_user()

for role in ['BIKE_ROLE', 'SNOW_ROLE', 'BIKE_SNOW_ROLE']:
    session.sql(f'grant role {role} to user {current_user}').collect()

Add permissions for those roles:


In [ ]:
USE ROLE ACCOUNTADMIN;

-- BIKE_ROLE:
GRANT OPERATE ON WAREHOUSE COMPUTE_WH TO ROLE BIKE_ROLE;
GRANT usage ON WAREHOUSE COMPUTE_WH TO ROLE BIKE_ROLE;

GRANT DATABASE ROLE SNOWFLAKE.CORTEX_USER TO ROLE BIKE_ROLE;

--- SNOW_ROLE:
GRANT OPERATE ON WAREHOUSE COMPUTE_WH TO ROLE SNOW_ROLE;
GRANT usage ON WAREHOUSE COMPUTE_WH TO ROLE SNOW_ROLE;

GRANT DATABASE ROLE SNOWFLAKE.CORTEX_USER TO ROLE SNOW_ROLE;

-- BIKE_SNOW_ROLE:

GRANT OPERATE ON WAREHOUSE COMPUTE_WH TO ROLE BIKE_SNOW_ROLE;
GRANT usage ON WAREHOUSE COMPUTE_WH TO ROLE BIKE_SNOW_ROLE;

GRANT DATABASE ROLE SNOWFLAKE.CORTEX_USER TO ROLE BIKE_SNOW_ROLE;


In [ ]:
session.use_role(this_session_role)


# Unstructured Data Processing
## Tools for Unstructured Data (Product Documentation)
We are going to setup a tool tha will help the agent to extract information from unstructured data. It will process PDF documents but also will be using IMAGE descriptions..The information is stored in PDF and JPEG format so first we are going to process it.

### PDF Files

We are going to process the documents about bikes and ski. We have copied them already from the Git repository into a Snowflake Stage area. Those file could also be located in an external S3 location, so there is no need to ingest them into Snowflake if they are already in a cloud location.


In [ ]:
-- Check what documents are available
ALTER STAGE docs REFRESH;
SELECT * FROM DIRECTORY('@DOCS');

Read the PDF files leveraging the [AI_PARSE_DOCUMENT](https://docs.snowflake.com/en/sql-reference/functions/ai_parse_document) function: 

In [ ]:
CREATE OR REPLACE TEMP TABLE RAW_TEXT AS
WITH FILE_TABLE as (
  (SELECT 
        RELATIVE_PATH,
        build_scoped_file_url(@docs, relative_path) as scoped_file_url,
        TO_FILE('@DOCS', RELATIVE_PATH) AS docs 
    FROM 
        DIRECTORY(@DOCS))
)
SELECT 
    RELATIVE_PATH,
    scoped_file_url,
    TO_VARCHAR (
        SNOWFLAKE.CORTEX.AI_PARSE_DOCUMENT (
            docs,
            {'mode': 'LAYOUT'} ):content
        ) AS EXTRACTED_LAYOUT 
FROM 
    FILE_TABLE
WHERE
    RELATIVE_PATH LIKE '%.pdf';

You can check the content

In [ ]:
select * from RAW_TEXT limit 5;

Create the table that will be used by Cortex Search Service as a Tool for Cortex Agents in order to retrieve information from PDF and JPEG files:

In [ ]:
create or replace TABLE DOCS_CHUNKS_TABLE ( 
    RELATIVE_PATH VARCHAR(16777216), -- Relative path to the PDF file
    scoped_file_url VARCHAR(16777216),
    CHUNK VARCHAR(16777216), -- Piece of text
    CHUNK_INDEX INTEGER, -- Index for the text
    USER_ROLE VARCHAR(16777216), -- Role that can access to this row
    CATEGORY VARCHAR(16777216)

);

Split the PDF file into pieces for better Retrieval. Here we can leverage the native [SPLIT_TEXT_RECURSIVE_CHARACTER](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex) function.

In [ ]:
insert into DOCS_CHUNKS_TABLE (relative_path, scoped_file_url, chunk, chunk_index)

    select relative_path, 
            scoped_file_url,
            c.value::TEXT as chunk,
            c.INDEX::INTEGER as chunk_index
            
    from 
        raw_text,
        LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
              EXTRACTED_LAYOUT,
              'markdown',
              1512,
              256,
              ['\n\n', '\n', ' ', '']
           )) c;


Check the content

In [ ]:
SELECT * FROM DOCS_CHUNKS_TABLE limit 7;

As a demo, we are goign to show how AI_CLASSIFY Cortex function to classify the document type. We have two classes, Bike and Snow, and we pass the document title and the first chunk of the document to the function, using it to classify the document.

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE docs_categories AS WITH unique_documents AS (
  SELECT
    DISTINCT relative_path, chunk
  FROM
    docs_chunks_table
  WHERE 
    chunk_index = 0
  ),
 docs_category_cte AS (
  SELECT
    relative_path,
    TRIM(snowflake.cortex.AI_CLASSIFY (
      'Title:' || relative_path || 'Content:' || chunk, ['Bike', 'Snow']
     )['labels'][0], '"') AS CATEGORY
  FROM
    unique_documents
)
SELECT
  *
FROM
  docs_category_cte;

We can check the classification is correct:

In [ ]:
select * from docs_categories;

Update the table with the classification label:

In [ ]:
UPDATE docs_chunks_table dct
SET
    category = dc.category,
    USER_ROLE = CASE
                    WHEN dc.CATEGORY = 'Bike' THEN 'BIKE_ROLE'
                    WHEN dc.CATEGORY = 'Snow' THEN 'SNOW_ROLE'
                    ELSE NULL -- Or a default role if categories other than 'Bike' or 'Snow' are possible
                END
FROM
    docs_categories dc
WHERE
    dct.relative_path = dc.relative_path;

In [ ]:
select * from docs_chunks_table limit 15;

### IMAGES

Now let's process the images we have for our bikes and skies. We are going to use [AI_COMPLETE](https://docs.snowflake.com/en/sql-reference/functions/ai_complete) and [AI_CLASSIFY](https://docs.snowflake.com/en/sql-reference/functions/ai_classify) multi-modal function asking for an image description and classification. We add it into the DOCS_CHUNKS_TABLE where we also have the PDF documentation. For AI_COMPLETE for Multi-Modal we are proposing claude-3-7-sonnet, but you should check what is the availability in your [region]( https://docs.snowflake.com/en/sql-reference/functions/ai_complete-single-file). 

We are goign to run the next cell first to enable [CROSS REGION INFERENCE](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cross-region-inference). In our case, this is running within AWS_EU region and we want to keep it there. 

In [ ]:
--ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_EU';
--ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_US';

In [ ]:
INSERT INTO DOCS_CHUNKS_TABLE (relative_path, scoped_file_url, chunk, chunk_index, category, USER_ROLE)
WITH classified_docs AS (
    SELECT
        RELATIVE_PATH,
        build_scoped_file_url(@docs, relative_path) as scoped_file_url,
        CONCAT('This is a picture describing: ' || RELATIVE_PATH ||
            ' .THIS IS THE DESCRIPTION: ' ||
            SNOWFLAKE.CORTEX.AI_COMPLETE('claude-3-7-sonnet',
            'This should be an image of a bike or snow gear. Provide all the detail that
            you can. Try to extract colors, brand names, parts, etc: ',
            TO_FILE('@DOCS', RELATIVE_PATH))) AS chunk,
        0 AS chunk_index,
        -- Calculate category once
        SNOWFLAKE.CORTEX.AI_CLASSIFY(
            TO_FILE('@DOCS', RELATIVE_PATH),
            ['Bike', 'Snow']):labels[0] AS category
    FROM
        DIRECTORY('@DOCS')
    WHERE
        RELATIVE_PATH LIKE '%.jpeg'
)
SELECT
    cd.relative_path,
    cd.scoped_file_url,
    cd.chunk,
    cd.chunk_index,
    cd.category,
    -- Use the calculated category to derive USER_ROLE
    CASE
        WHEN cd.category LIKE '%Bike%' THEN 'BIKE_ROLE'
        WHEN cd.category LIKE '%Snow%' THEN 'SNOW_ROLE'
        ELSE NULL -- Or a default role if categories other than 'Bike' or 'Snow' are possible
    END AS USER_ROLE
FROM
    classified_docs cd;

You can check the descriptions:

In [ ]:
select * from DOCS_CHUNKS_TABLE
    where RELATIVE_PATH LIKE '%.jpeg';

At this point we should have only 2 roles. Check:

In [ ]:
select distinct(USER_ROLE) from DOCS_CHUNKS_TABLE;

In [ ]:
current_user = session.get_current_user()

for role in ['BIKE_ROLE', 'SNOW_ROLE', 'BIKE_SNOW_ROLE']:
    session.sql(f'grant role {role} to user {current_user}').collect()

## Enable Cortex Search to be used by the agent for Retrieval

Cortex Search is the Tool that will be used to retrieve context from unstructured data. Once we have processed all the content from PDF and IMAGES into the DOCS_CHUNK_TABLE, we just need to enable the service in that table. This will create the embeddings, indexing, etc automatically. 

We are going to create one service that will index all documents, other for Bikes, that will be leveraged by agents with the Bike role and other for Snow.

There is no need to have separate tables as we al filtering when creating each service.

In [ ]:
create or replace CORTEX SEARCH SERVICE DOCUMENTATION_TOOL
ON chunk
ATTRIBUTES category, user_role
warehouse = COMPUTE_WH
TARGET_LAG = '1 hour'
EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
as (
    select chunk,
        chunk_index,
        relative_path,
        scoped_file_url,
        category,
        user_role
    from docs_chunks_table
);

In [ ]:
create or replace CORTEX SEARCH SERVICE DOCUMENTATION_TOOL_BIKES
ON chunk
ATTRIBUTES category, user_role
warehouse = COMPUTE_WH
TARGET_LAG = '1 hour'
EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
as (
    select chunk,
        chunk_index,
        relative_path,
        scoped_file_url,
        category,
        user_role
    from docs_chunks_table
    where user_role = 'BIKE_ROLE'
);

In [ ]:
create or replace CORTEX SEARCH SERVICE DOCUMENTATION_TOOL_SNOW
ON chunk
ATTRIBUTES category, user_role
warehouse = COMPUTE_WH
TARGET_LAG = '1 hour'
EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
as (
    select chunk,
        chunk_index,
        relative_path,
        scoped_file_url,
        category,
        user_role
    from docs_chunks_table
    where user_role = 'SNOW_ROLE'
);

The Role needs to have access to the Cortex Search Service. We are grant it based on each Role:

In [ ]:
USE ROLE ACCOUNTADMIN;

-- BIKE_ROLE:

GRANT USAGE ON CORTEX SEARCH SERVICE DOCUMENTATION_TOOL_BIKES TO ROLE BIKE_ROLE;

--- SNOW_ROLE:

GRANT USAGE ON CORTEX SEARCH SERVICE DOCUMENTATION_TOOL_SNOW TO ROLE SNOW_ROLE;

-- BIKE_SNOW_ROLE:

GRANT USAGE ON CORTEX SEARCH SERVICE DOCUMENTATION_TOOL TO ROLE BIKE_SNOW_ROLE;

In [ ]:
session.use_role(this_session_role)


## Tools for Unstructured Data (Customer Reviews)

In the previous step we have processed Unstructured data in the format of PDF files. Next we are going to process the customer reviews that are already located in a Snowflake table within a column. We will enable Cortex Search so we can create a tool for our agents that will retreive customer reviews:


In [ ]:
CREATE OR REPLACE TABLE customer_experience_comments (
    id INT,
    experience_date DATE,
    product_id INT,
    product_name VARCHAR(100),
    comment TEXT
);

COPY INTO customer_experience_comments (
    id,
    experience_date,
    product_id,
    product_name,
    comment
)
FROM @csv/customer_experience_comments.csv
FILE_FORMAT = (
    TYPE = 'CSV'
    FIELD_DELIMITER = ','
    RECORD_DELIMITER = '\n'
    SKIP_HEADER = 1
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    NULL_IF = ('NULL', 'null', '')
    ERROR_ON_COLUMN_COUNT_MISMATCH = TRUE
);

In [ ]:
select * from customer_experience_comments limit 10;

Create the Cortex Search Service for the customer experience comments. We get the date, the product name and the comments and provide it as the Index for Cortex Search.

This will be provided as a Tool for the agent when having to retriveve customer comments for products. 

In [ ]:
create or replace CORTEX SEARCH SERVICE CUSTOMER_EXPERIENCE_TOOL
ON product_experience
warehouse = COMPUTE_WH
TARGET_LAG = '1 hour'
EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
as (
    select experience_date, product_id, product_name, comment,
    CONCAT(experience_date, ' ', product_name, ': ', comment) AS product_experience
    from customer_experience_comments
);

# Tools for Structured Data (Cortex Analyst)

Another Tool that we will be providing to the Cortex Agent will be Cortex Analyst which will provide the capability to extract information from Snowflake Tables.  In the API call we will be providing the location of a Semantic file that contains information about the business terminology to describe the data.

First we are going to create some syntethic data about the bikes and ski products that we have.

We are going to create the following tables with content:


**DIM_ARTICLE – Article/Item Dimension**

Purpose: Stores descriptive information about the products (articles) being sold.

Key Columns:

- ARTICLE_ID (Primary Key): Unique identifier for each article.
- ARTICLE_NAME: Full name/description of the product.
- ARTICLE_CATEGORY: Product category (e.g., Bike, Skis, Ski Boots).
- ARTICLE_BRAND: Manufacturer or brand (e.g., Mondracer, Carver).
- ARTICLE_COLOR: Dominant color for the article.
- ARTICLE_PRICE: Standard unit price of the article.


**DIM_CUSTOMER – Customer Dimension**

Purpose: Contains demographic and segmentation info about each customer.

Key Columns:

- CUSTOMER_ID (Primary Key): Unique identifier for each customer.
- CUSTOMER_NAME: Display name for the customer.
- CUSTOMER_REGION: Geographic region (e.g., North, South).
- CUSTOMER_AGE: Age of the customer.
- CUSTOMER_GENDER: Gender (Male/Female).
- CUSTOMER_SEGMENT: Marketing segment (e.g., Premium, Regular, Occasional).


**FACT_SALES – Sales Transactions Fact Table**

Purpose: Captures individual sales transactions (facts) with references to article and customer details.

Key Columns:

- SALE_ID (Primary Key): Unique identifier for the transaction.
- ARTICLE_ID (Foreign Key): Links to DIM_ARTICLE.
- CUSTOMER_ID (Foreign Key): Links to DIM_CUSTOMER.
- DATE_SALES: Date when the sale occurred.
- QUANTITY_SOLD: Number of units sold in the transaction.
- TOTAL_PRICE: Total transaction value (unit price × quantity).
- SALES_CHANNEL: Sales channel used (e.g., Online, In-Store, Partner).
- PROMOTION_APPLIED: Boolean indicating if the sale involved a promotion or discount.


In [ ]:
CREATE OR REPLACE TABLE DIM_ARTICLE (
    ARTICLE_ID INT PRIMARY KEY,
    ARTICLE_NAME STRING,
    ARTICLE_CATEGORY STRING,
    ARTICLE_BRAND STRING,
    ARTICLE_COLOR STRING,
    ARTICLE_PRICE FLOAT
);


INSERT INTO DIM_ARTICLE (ARTICLE_ID, ARTICLE_NAME, ARTICLE_CATEGORY, ARTICLE_BRAND, ARTICLE_COLOR, ARTICLE_PRICE)
VALUES 
(1, 'Mondracer Infant Bike', 'Bike', 'Mondracer', 'Red', 3000),
(2, 'Premium Bicycle', 'Bike', 'Veloci', 'Blue', 9000),
(3, 'Ski Boots TDBootz Special', 'Ski Boots', 'TDBootz', 'Black', 600),
(4, 'The Ultimate Downhill Bike', 'Bike', 'Graviton', 'Green', 10000),
(5, 'The Xtreme Road Bike 105 SL', 'Bike', 'Xtreme', 'White', 8500),
(6, 'Carver Skis', 'Skis', 'Carver', 'Orange', 790),
(7, 'Outpiste Skis', 'Skis', 'Outpiste', 'Yellow', 900),
(8, 'Racing Fast Skis', 'Skis', 'RacerX', 'Blue', 950);
     

We are going to use [Row Access Policies](https://docs.snowflake.com/en/user-guide/security-row-intro) in order to define what rows can be retrieved for each of the roles. These policies will be respected by the Agents that will be quering the sales tables. 

In our case, the BIKE_ROLE will be able to see the rows containing bike products and the SNOW_ROLE will be able to access Snow articles.

First we define a tabe with the mapping of who can do what:

In [ ]:
CREATE OR REPLACE TABLE ROW_ACCESS_MAPPING(
    ALLOWED_CATEGORY STRING,
    ALLOWED_ROLE STRING
);

INSERT INTO ROW_ACCESS_MAPPING (ALLOWED_CATEGORY, ALLOWED_ROLE)
VALUES
('Bike', 'BIKE_ROLE'),
('Ski Boots', 'SNOW_ROLE'),
('Skis', 'SNOW_ROLE'),
('Bike', 'BIKE_SNOW_ROLE'),
('Ski Boots', 'BIKE_SNOW_ROLE'),
('Skis', 'BIKE_SNOW_ROLE')
;

Define the Row Access Policy using the mapping we have defined:

In [ ]:
CREATE OR REPLACE ROW ACCESS POLICY categories_policy
AS (ARTICLE_CATEGORY varchar) RETURNS BOOLEAN ->
    'ACCOUNTADMIN' = current_role()
    or exists(
        select * from ROW_ACCESS_MAPPING
            where allowed_role = current_role()
            and allowed_category = ARTICLE_CATEGORY
    );

In [ ]:
CREATE OR REPLACE ROW ACCESS POLICY categories_policy
AS (ARTICLE_CATEGORY varchar) RETURNS BOOLEAN ->
    'ACCOUNTADMIN' = current_role()
    or 'SPCS_PSE_ROLE' = current_role()
    or exists(
        select * from ROW_ACCESS_MAPPING
            where allowed_role = current_role()
            and allowed_category = ARTICLE_CATEGORY
    );

Apply that policy on the DIM_ARTICE table:

In [ ]:
ALTER TABLE DIM_ARTICLE
  ADD ROW ACCESS POLICY categories_policy
  ON (ARTICLE_CATEGORY);

In [ ]:
CREATE OR REPLACE TABLE DIM_CUSTOMER (
    CUSTOMER_ID INT PRIMARY KEY,
    CUSTOMER_NAME STRING,
    CUSTOMER_REGION STRING,
    CUSTOMER_AGE INT,
    CUSTOMER_GENDER STRING,
    CUSTOMER_SEGMENT STRING
);

INSERT INTO DIM_CUSTOMER (CUSTOMER_ID, CUSTOMER_NAME, CUSTOMER_REGION, CUSTOMER_AGE, CUSTOMER_GENDER, CUSTOMER_SEGMENT)
SELECT 
    SEQ4() AS CUSTOMER_ID,
    'Customer ' || SEQ4() AS CUSTOMER_NAME,
    CASE MOD(SEQ4(), 5)
        WHEN 0 THEN 'North'
        WHEN 1 THEN 'South'
        WHEN 2 THEN 'East'
        WHEN 3 THEN 'West'
        ELSE 'Central'
    END AS CUSTOMER_REGION,
    UNIFORM(18, 65, RANDOM()) AS CUSTOMER_AGE,
    CASE MOD(SEQ4(), 2)
        WHEN 0 THEN 'Male'
        ELSE 'Female'
    END AS CUSTOMER_GENDER,
    CASE MOD(SEQ4(), 3)
        WHEN 0 THEN 'Premium'
        WHEN 1 THEN 'Regular'
        ELSE 'Occasional'
    END AS CUSTOMER_SEGMENT
FROM TABLE(GENERATOR(ROWCOUNT => 5000));

The Git repository contains a CSV file with sales information that we are going to use. We had already copy it into a Snowflaek Staging area


In [ ]:
CREATE OR REPLACE TABLE FACT_SALES (
    SALE_ID INT PRIMARY KEY,
    ARTICLE_ID INT,
    DATE_SALES DATE,
    CUSTOMER_ID INT,
    QUANTITY_SOLD INT,
    TOTAL_PRICE FLOAT,
    SALES_CHANNEL STRING,
    PROMOTION_APPLIED BOOLEAN,
    FOREIGN KEY (ARTICLE_ID) REFERENCES DIM_ARTICLE(ARTICLE_ID),
    FOREIGN KEY (CUSTOMER_ID) REFERENCES DIM_CUSTOMER(CUSTOMER_ID)
);

COPY INTO FACT_SALES (
    SALE_ID,
    ARTICLE_ID, 
    DATE_SALES,
    CUSTOMER_ID,
    QUANTITY_SOLD,
    TOTAL_PRICE,
    SALES_CHANNEL,
    PROMOTION_APPLIED
)
FROM @csv/fact_sales.csv
FILE_FORMAT = (
    TYPE = 'CSV'
    FIELD_DELIMITER = ','
    RECORD_DELIMITER = '\n'
    SKIP_HEADER = 1
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    NULL_IF = ('NULL', 'null', '')
    ERROR_ON_COLUMN_COUNT_MISMATCH = TRUE
);

In [ ]:
select * from FACT_SALES limit 3;

## Improving Tool Usage with Dynamic Literal Retrieval

Thanks to the Cortex Analyst integration with Cortex Search, we can improve the retrieval of all possible values of a column. Instead of listing all the possible values in the semantic file, we can use Cortex Search Integration. 

Let's use it as example for the ARTICLE NAMES, so we do not have to list all.


In [ ]:
CREATE OR REPLACE TABLE ARTICLE_NAMES AS
  SELECT
      DISTINCT ARTICLE_NAME AS ARTICLE_NAME
  FROM DIM_ARTICLE;

CREATE OR REPLACE CORTEX SEARCH SERVICE _ARTICLE_NAME_SEARCH
  ON ARTICLE_NAME
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
       ARTICLE_NAME
  FROM ARTICLE_NAMES
);

In [ ]:
GRANT USAGE ON CORTEX SEARCH SERVICE _ARTICLE_NAME_SEARCH TO ROLE SNOW_ROLE;
GRANT USAGE ON CORTEX SEARCH SERVICE _ARTICLE_NAME_SEARCH TO ROLE BIKE_ROLE;
GRANT USAGE ON CORTEX SEARCH SERVICE _ARTICLE_NAME_SEARCH TO ROLE BIKE_SNOW_ROLE;

## Semantic Model Definition for Cortex Analyst

[Semantic models](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst/semantic-model-spec) map business terminology to database schemas and add contextual meaning. Here we define the tables, their relationships, and the description for their dimensions and facts. 

We can also produce specific instructions that will help Cortex Analyst to provide the right SQL.

Cortex Analyst will be used by the agents as the Tool to retrieve information from Snowflake Table that needs to be extracted using SQL.




In [ ]:
create or replace semantic view SALES_DATA_SEMANTIC_VIEW
	tables (
		CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.DIM_ARTICLE primary key (ARTICLE_ID),
		CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.DIM_CUSTOMER primary key (CUSTOMER_ID),
		CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.FACT_SALES primary key (CUSTOMER_ID,ARTICLE_ID)
	)
	relationships (
		ARTICLE_JOIN as FACT_SALES(ARTICLE_ID) references DIM_ARTICLE(ARTICLE_ID),
		CUSTOMER_JOIN as FACT_SALES(CUSTOMER_ID) references DIM_CUSTOMER(CUSTOMER_ID)
	)
	facts (
		DIM_ARTICLE.ARTICLE_ID as ARTICLE_ID with synonyms=('article_key','article_reference','item_id','item_number','product_code','product_id') comment='Unique identifier for an article in the inventory.',
		DIM_ARTICLE.ARTICLE_PRICE as ARTICLE_PRICE with synonyms=('article_value','item_cost','list_price','product_price','retail_price','sale_price','unit_price') comment='The price of an article, representing the monetary value assigned to a specific article or product.',
		DIM_CUSTOMER.CUSTOMER_AGE as CUSTOMER_AGE with synonyms=('age_of_customer','customer_birth_year','customer_dob','customer_maturity','years_old') comment='The age of the customer at the time of data collection, representing the number of years since birth.',
		DIM_CUSTOMER.CUSTOMER_ID as CUSTOMER_ID with synonyms=('account_number','client_id','client_identifier','customer_key','patron_id','subscriber_id','user_id') comment='Unique identifier for each customer in the database.',
		FACT_SALES.ARTICLE_ID as ARTICLE_ID with synonyms=('article_number','item_code','item_id','item_number','product_code','product_id','product_number') comment='Unique identifier for the article being sold.',
		FACT_SALES.CUSTOMER_ID as CUSTOMER_ID with synonyms=('account_id','buyer_id','client_id','client_number','customer_number','patron_id','user_id') comment='Unique identifier for the customer who made the sale.',
		FACT_SALES.QUANTITY_SOLD as QUANTITY_SOLD with synonyms=('amount_sold','items_sold','number_sold','quantity_purchased','sales_volume','sold_quantity','units_sold') comment='The total number of units of a product sold in a single transaction.',
		FACT_SALES.SALE_ID as SALE_ID with synonyms=('invoice_number','order_id','purchase_id','sale_number','sale_reference','sales_transaction','transaction_id','transaction_reference') comment='Unique identifier for each sales transaction.',
		FACT_SALES.TOTAL_PRICE as TOTAL_PRICE with synonyms=('sale_amount','total_amount','total_cost','total_invoice_value','total_revenue','total_sale_value','total_transaction_value') comment='The total price of a sale, representing the overall amount paid by a customer for a particular transaction.'
	)
	dimensions (
		DIM_ARTICLE.ARTICLE_BRAND as ARTICLE_BRAND with synonyms=('article_label','article_maker','brand_name','item_brand','manufacturer','product_brand') comment='The brand name of the article, representing the manufacturer or label associated with the product.',
		DIM_ARTICLE.ARTICLE_CATEGORY as ARTICLE_CATEGORY with synonyms=('article_grouping','article_type','category_name','classification','genre','item_group','product_category','product_class') comment='The category of the article, such as Bike, Ski Boots, or Skis, which represents the type of product being sold or managed.',
		DIM_ARTICLE.ARTICLE_COLOR as ARTICLE_COLOR with synonyms=('article_hue','article_shade','color','hue','item_color','product_color','shade','tint') comment='The color of the article, which can be one of the following: Red, Blue, or Black.',
		DIM_ARTICLE.ARTICLE_NAME as ARTICLE_NAME with synonyms=('article_title','item_description','item_name','product_description','product_name','product_title') comment='The name of the article or product being sold, such as a specific type of bicycle.' with cortex search service _ARTICLE_NAME_SEARCH,
		DIM_CUSTOMER.CUSTOMER_GENDER as CUSTOMER_GENDER with synonyms=('customer_sex','demographic_gender','gender_category','gender_type','sex') comment='The gender of the customer, either Male or Female.',
		DIM_CUSTOMER.CUSTOMER_NAME as CUSTOMER_NAME with synonyms=('account_holder','account_name','buyer_name','client_name','client_title','customer_title','patron_name') comment='The name of the customer, used to identify and distinguish between individual customers.',
		DIM_CUSTOMER.CUSTOMER_REGION as CUSTOMER_REGION with synonyms=('area_code','customer_location','customer_territory','geographic_area','market_area','region_code','sales_region','territory') comment='Geographic region where the customer is located.',
		DIM_CUSTOMER.CUSTOMER_SEGMENT as CUSTOMER_SEGMENT with synonyms=('client_segment','customer_category','customer_group','customer_type','demographic_segment','market_segment','patron_category') comment='The CUSTOMER_SEGMENT dimension categorizes customers based on their purchase behavior and loyalty, with three distinct segments: Premium (high-value, frequent customers), Regular (consistent, mid-value customers), and Occasional (infrequent, low-value customers).',
		FACT_SALES.PROMOTION_APPLIED as PROMOTION_APPLIED with synonyms=('discount_flag','discount_used','offer_applied','promo_used','promotion_used','sale_applied','sale_flag','special_offer') comment='Indicates whether a promotion was applied to a sale.',
		FACT_SALES.SALES_CHANNEL as SALES_CHANNEL with synonyms=('distribution_channel','point_of_sale','sales_medium','sales_medium_type','sales_outlet','sales_route','sales_source') comment='The sales channel through which the sale was made, indicating whether the transaction occurred online, in a physical store, or through a business partner.',
		FACT_SALES.DATE_SALES as DATE_SALES with synonyms=('ORDER_DATE','PURCHASE_DATE','SALE_TIMESTAMP','SALES_DATE','TRANSACTION_DATE') comment='Date on which the sales transaction occurred.'
	)
	comment='The data model consists of three core tables: DIM_ARTICLE, which stores detailed information about the products being sold, including their name, category, brand, color, and price; DIM_CUSTOMER, which contains customer profiles with attributes like region, age, gender, and marketing segment; and FACT_SALES, which captures all sales transactions by linking customers to purchased articles, along with details such as sale date, quantity, total price, sales channel, and whether a promotion was applied. Together, these tables provide a comprehensive view of products, customers, and sales activity, enabling rich analysis across multiple dimensions.'
	ai_sql_generation 'Limit answer to sales questions about products. You should not answer questions about product specifications or usage.'
	with extension (CA='{"tables":[{"name":"DIM_ARTICLE","dimensions":[{"name":"ARTICLE_BRAND","sample_values":["Mondracer","Veloci","TDBootz"]},{"name":"ARTICLE_CATEGORY","sample_values":["Bike","Ski Boots","Skis"]},{"name":"ARTICLE_COLOR","sample_values":["Red","Blue","Black"]},{"name":"ARTICLE_NAME"}],"facts":[{"name":"ARTICLE_ID","sample_values":["1","2","3"]},{"name":"ARTICLE_PRICE","sample_values":["3000","9000","10000"]}]},{"name":"DIM_CUSTOMER","dimensions":[{"name":"CUSTOMER_GENDER","sample_values":["Male","Female"]},{"name":"CUSTOMER_NAME","sample_values":["Customer 0","Customer 1","Customer 2"]},{"name":"CUSTOMER_REGION","sample_values":["North","South","East"]},{"name":"CUSTOMER_SEGMENT","sample_values":["Premium","Regular","Occasional"]}],"facts":[{"name":"CUSTOMER_AGE","sample_values":["38","23","24"]},{"name":"CUSTOMER_ID","sample_values":["0","1","2"]}]},{"name":"FACT_SALES","dimensions":[{"name":"PROMOTION_APPLIED","sample_values":["TRUE","FALSE"]},{"name":"SALES_CHANNEL","sample_values":["Online","In-Store","Partner"]}],"facts":[{"name":"ARTICLE_ID","sample_values":["5","4","7"]},{"name":"CUSTOMER_ID","sample_values":["3678","3031","1927"]},{"name":"QUANTITY_SOLD","sample_values":["9","2","10"]},{"name":"SALE_ID","sample_values":["0","1","2"]},{"name":"TOTAL_PRICE","sample_values":["76500","100000","10000"]}],"time_dimensions":[{"name":"DATE_SALES","sample_values":["2022-04-16","2022-04-17","2022-04-18"]}]}],"relationships":[{"name":"customer_join","relationship_type":"many_to_one","join_type":"inner"},{"name":"article_join","relationship_type":"many_to_one","join_type":"inner"}]}');

The Semantic View Spec can be extracted and used for automatic definition:

In [ ]:
call get_ddl ('SEMANTIC VIEW', 'SALES_DATA_SEMANTIC_VIEW')

## Role Based Access Control

We have defined previously 3 different roles. Now we are goign to assign the specific objects needed by each role:

In [ ]:
-- BIKE_ROLE:

GRANT USAGE ON DATABASE CC_SNOWFLAKE_INTELLIGENCE_E2E TO ROLE BIKE_ROLE;
GRANT USAGE ON SCHEMA PUBLIC TO ROLE BIKE_ROLE;
GRANT SELECT ON TABLE DIM_ARTICLE TO ROLE BIKE_ROLE;
GRANT SELECT ON TABLE DIM_CUSTOMER TO ROLE BIKE_ROLE;
GRANT SELECT ON TABLE FACT_SALES TO ROLE BIKE_ROLE;

--- SNOW_ROLE:

GRANT USAGE ON DATABASE CC_SNOWFLAKE_INTELLIGENCE_E2E TO ROLE SNOW_ROLE;
GRANT USAGE ON SCHEMA PUBLIC TO ROLE SNOW_ROLE;
GRANT SELECT ON TABLE DIM_ARTICLE TO ROLE SNOW_ROLE;
GRANT SELECT ON TABLE DIM_CUSTOMER TO ROLE SNOW_ROLE;
GRANT SELECT ON TABLE FACT_SALES TO ROLE SNOW_ROLE;

-- BIKE_SNOW_ROLE:

GRANT USAGE ON DATABASE CC_SNOWFLAKE_INTELLIGENCE_E2E TO ROLE BIKE_SNOW_ROLE;
GRANT USAGE ON SCHEMA PUBLIC TO ROLE BIKE_SNOW_ROLE;
GRANT SELECT ON TABLE DIM_ARTICLE TO ROLE BIKE_SNOW_ROLE;
GRANT SELECT ON TABLE DIM_CUSTOMER TO ROLE BIKE_SNOW_ROLE;
GRANT SELECT ON TABLE FACT_SALES TO ROLE BIKE_SNOW_ROLE;


In [ ]:
session.sql(f"GRANT USAGE ON DATABASE CC_SNOWFLAKE_INTELLIGENCE_E2E TO ROLE {this_session_role};").collect()
session.sql(f"GRANT USAGE ON SCHEMA PUBLIC TO ROLE {this_session_role};").collect()
session.sql(f"GRANT SELECT ON TABLE DIM_ARTICLE TO ROLE {this_session_role};").collect()
session.sql(f"GRANT SELECT ON TABLE DIM_CUSTOMER TO ROLE {this_session_role};").collect()
session.sql(f"GRANT SELECT ON TABLE FACT_SALES TO ROLE {this_session_role};").collect()


Test how RBAC works. Different roles will be able to see different analytical data.

When using the SNOW_ROLE, the query should only retrieve Snow related products:

In [ ]:
USE ROLE SNOW_ROLE;
WITH yearly_sales AS (
  SELECT
    f.article_id,
    SUM(f.total_price) AS total_sales
  FROM
    fact_sales AS f
  WHERE
    DATE_PART('year', f.date_sales) = 2025
  GROUP BY
    f.article_id
)
SELECT
  a.article_name,
  a.article_category,
  a.article_brand,
  ys.total_sales
FROM
  yearly_sales AS ys
  INNER JOIN dim_article AS a ON ys.article_id = a.article_id
ORDER BY
  ys.total_sales DESC NULLS LAST

Exact same query, but run by the BIKE_ROLE should retrieve only Bike related products:

In [ ]:
USE ROLE BIKE_ROLE;
WITH yearly_sales AS (
  SELECT
    f.article_id,
    SUM(f.total_price) AS total_sales
  FROM
    fact_sales AS f
  WHERE
    DATE_PART('year', f.date_sales) = 2025
  GROUP BY
    f.article_id
)
SELECT
  a.article_name,
  a.article_category,
  a.article_brand,
  ys.total_sales
FROM
  yearly_sales AS ys
  INNER JOIN dim_article AS a ON ys.article_id = a.article_id
ORDER BY
  ys.total_sales DESC NULLS LAST

In [ ]:
session.use_role(this_session_role)

# Snowflake Intelligence Setup - Agents Definitions

You can either define your Agents and tools using Snowsight or programaticaly. We will use the [CREATE AGENT](https://docs.snowflake.com/en/sql-reference/sql/create-agent) statement as part of this notebook. 

The BIKES_SNOW_AGENT will leverage the Cortex Search Service DOCUMENTATION_TOOL that will be able to retrieve documents for both bikes and snow articles


In [ ]:
CREATE OR REPLACE AGENT BIKES_SNOW_AGENT
COMMENT = "Bikes & Snow Products and Sales Agent"
FROM SPECIFICATION
$$
{"models":{"orchestration":"auto"},

"orchestration":{},"instructions":{"response":"Be always polite and answer asking if they need further help","orchestration":"Your Role: You are \"SalesExpertBot\", a sales intelligence assistant for the retailer agents. Your Scope: You answer questions about product specifications and sales for those products. Your role is also help sales to understand how product sales are evolving and how differences in those products can affect their sales numbers.\n\nWhen answering product specifications always check you are answering the question for the product the question is asked for. You should look to the document title to identify documents properly.\n","sample_questions":[{"question":"What is the guarantee of the Premium bike?"},{"question":"What is the bike with most sales revenue during last year and what is their guarantee?"},{"question":"What is the monthly performance sales for the xtreme road bike"},{"question":"When the xtreme road bike sales started declining and what could be the reason for that?"}]},

"tools":[{"tool_spec":{"type":"cortex_analyst_text_to_sql","name":"SALES_DATA_SEMANTIC_VIEW","description":"REASONING:\nThis semantic view represents a classic star schema retail analytics model focused on sales performance analysis. The three tables form a cohesive data model where FACT_SALES serves as the central transaction table connected to two dimension tables through foreign key relationships. DIM_ARTICLE provides product context while DIM_CUSTOMER provides buyer context, enabling comprehensive analysis of who bought what, when, where, and how much. The relationships allow for multi-dimensional analysis across product categories, customer segments, sales channels, and time periods.\n\nDESCRIPTION:\nThe SALES_DATA_SEMANTIC_VIEW provides a comprehensive retail sales analytics framework built on three interconnected tables in the CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC schema. The central FACT_SALES table captures all sales transactions with details like quantities, prices, channels, and promotions, while connecting to DIM_ARTICLE (product catalog with brands, categories, colors, and prices) and DIM_CUSTOMER (customer profiles with demographics, regions, and segments) through many-to-one relationships. This star schema design enables rich multi-dimensional analysis of sales performance across product attributes, customer characteristics, sales channels, and time periods. The model supports comprehensive business intelligence queries for revenue analysis, customer segmentation, product performance, and promotional effectiveness across the company's retail operations."}},

{"tool_spec":{"type":"cortex_search","name":"DOCUMENTATION_TOOL","description":"Tool to access product specifications. "}},{"tool_spec":{"type":"cortex_search","name":"CUSTOMER_EXPERIENCE","description":"This tool provides access to the comments and feedback customers provide about products"}}],"tool_resources":{"CUSTOMER_EXPERIENCE":{"id_column":"PRODUCT_NAME","max_results":10,"search_service":"CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.CUSTOMER_EXPERIENCE_TOOL","title_column":"PRODUCT_NAME"},"DOCUMENTATION_TOOL":{"id_column":"SCOPED_FILE_URL","max_results":6,"search_service":"CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.DOCUMENTATION_TOOL","title_column":"RELATIVE_PATH"},"SALES_DATA_SEMANTIC_VIEW":{"execution_environment":{"type":"warehouse","warehouse":""},"semantic_view":"CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.SALES_DATA_SEMANTIC_VIEW"}}}
$$;


Grant usage for this Agent to the specific BIKE_SNOW_ROLE role:

In [ ]:
GRANT USAGE ON AGENT BIKES_SNOW_AGENT to role BIKE_SNOW_ROLE;

In [ ]:
--ALTER SNOWFLAKE INTELLIGENCE SNOWFLAKE_INTELLIGENCE_OBJECT_DEFAULT ADD AGENT BIKES_SNOW_AGENT;

The BIKES_AGENT will leverage the Cortex Search Service DOCUMENTATION_TOOL_BIKES that will be able to retrieve documents for both bikes and snow articles


In [ ]:
CREATE OR REPLACE AGENT BIKES_AGENT
COMMENT = "Bikes Products and Sales Agent"
FROM SPECIFICATION
$$
{"models":{"orchestration":"auto"},

"orchestration":{},"instructions":{"response":"Be always polite and answer asking if they need further help","orchestration":"Your Role: You are \"SalesExpertBot\", a sales intelligence assistant for the retailer agents. Your Scope: You answer questions about product specifications and sales for those products. Your role is also help sales to understand how product sales are evolving and how differences in those products can affect their sales numbers.\n\nWhen answering product specifications always check you are answering the question for the product the question is asked for. You should look to the document title to identify documents properly.\n","sample_questions":[{"question":"What is the guarantee of the Premium bike?"},{"question":"What is the bike with most sales revenue during last year and what is their guarantee?"},{"question":"What is the monthly performance sales for the xtreme road bike"},{"question":"When the xtreme road bike sales started declining and what could be the reason for that?"}]},

"tools":[{"tool_spec":{"type":"cortex_analyst_text_to_sql","name":"SALES_DATA_SEMANTIC_VIEW","description":"REASONING:\nThis semantic view represents a classic star schema retail analytics model focused on sales performance analysis. The three tables form a cohesive data model where FACT_SALES serves as the central transaction table connected to two dimension tables through foreign key relationships. DIM_ARTICLE provides product context while DIM_CUSTOMER provides buyer context, enabling comprehensive analysis of who bought what, when, where, and how much. The relationships allow for multi-dimensional analysis across product categories, customer segments, sales channels, and time periods.\n\nDESCRIPTION:\nThe SALES_DATA_SEMANTIC_VIEW provides a comprehensive retail sales analytics framework built on three interconnected tables in the CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC schema. The central FACT_SALES table captures all sales transactions with details like quantities, prices, channels, and promotions, while connecting to DIM_ARTICLE (product catalog with brands, categories, colors, and prices) and DIM_CUSTOMER (customer profiles with demographics, regions, and segments) through many-to-one relationships. This star schema design enables rich multi-dimensional analysis of sales performance across product attributes, customer characteristics, sales channels, and time periods. The model supports comprehensive business intelligence queries for revenue analysis, customer segmentation, product performance, and promotional effectiveness across the company's retail operations."}},

{"tool_spec":{"type":"cortex_search","name":"DOCUMENTATION_TOOL_BIKES","description":"Tool to access product specifications. "}},{"tool_spec":{"type":"cortex_search","name":"CUSTOMER_EXPERIENCE","description":"This tool provides access to the comments and feedback customers provide about products"}}],"tool_resources":{"CUSTOMER_EXPERIENCE":{"id_column":"PRODUCT_NAME","max_results":10,"search_service":"CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.CUSTOMER_EXPERIENCE_TOOL","title_column":"PRODUCT_NAME"},"DOCUMENTATION_TOOL_BIKES":{"id_column":"SCOPED_FILE_URL","max_results":6,"search_service":"CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.DOCUMENTATION_TOOL_BIKES","title_column":"RELATIVE_PATH"},"SALES_DATA_SEMANTIC_VIEW":{"execution_environment":{"type":"warehouse","warehouse":""},"semantic_view":"CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.SALES_DATA_SEMANTIC_VIEW"}}}
$$;


In [ ]:
GRANT USAGE ON AGENT BIKES_AGENT to role BIKE_ROLE;

The SNOW_AGENT will leverage the Cortex Search Service DOCUMENTATION_TOOL_SNOW that will be able to retrieve documents for both bikes and snow articles


In [ ]:
CREATE OR REPLACE AGENT SNOW_AGENT
COMMENT = "Snow Products and Sales Agent"
FROM SPECIFICATION
$$
{"models":{"orchestration":"auto"},

"orchestration":{},"instructions":{"response":"Be always polite and answer asking if they need further help","orchestration":"Your Role: You are \"SalesExpertBot\", a sales intelligence assistant for the retailer agents. Your Scope: You answer questions about product specifications and sales for those products. Your role is also help sales to understand how product sales are evolving and how differences in those products can affect their sales numbers.\n\nWhen answering product specifications always check you are answering the question for the product the question is asked for. You should look to the document title to identify documents properly.\n","sample_questions":[{"question":"What is the guarantee of the Premium bike?"},{"question":"What is the bike with most sales revenue during last year and what is their guarantee?"},{"question":"What is the monthly performance sales for the xtreme road bike"},{"question":"When the xtreme road bike sales started declining and what could be the reason for that?"}]},

"tools":[{"tool_spec":{"type":"cortex_analyst_text_to_sql","name":"SALES_DATA_SEMANTIC_VIEW","description":"REASONING:\nThis semantic view represents a classic star schema retail analytics model focused on sales performance analysis. The three tables form a cohesive data model where FACT_SALES serves as the central transaction table connected to two dimension tables through foreign key relationships. DIM_ARTICLE provides product context while DIM_CUSTOMER provides buyer context, enabling comprehensive analysis of who bought what, when, where, and how much. The relationships allow for multi-dimensional analysis across product categories, customer segments, sales channels, and time periods.\n\nDESCRIPTION:\nThe SALES_DATA_SEMANTIC_VIEW provides a comprehensive retail sales analytics framework built on three interconnected tables in the CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC schema. The central FACT_SALES table captures all sales transactions with details like quantities, prices, channels, and promotions, while connecting to DIM_ARTICLE (product catalog with brands, categories, colors, and prices) and DIM_CUSTOMER (customer profiles with demographics, regions, and segments) through many-to-one relationships. This star schema design enables rich multi-dimensional analysis of sales performance across product attributes, customer characteristics, sales channels, and time periods. The model supports comprehensive business intelligence queries for revenue analysis, customer segmentation, product performance, and promotional effectiveness across the company's retail operations."}},

{"tool_spec":{"type":"cortex_search","name":"DOCUMENTATION_TOOL_SNOW","description":"Tool to access product specifications. "}},{"tool_spec":{"type":"cortex_search","name":"CUSTOMER_EXPERIENCE","description":"This tool provides access to the comments and feedback customers provide about products"}}],"tool_resources":{"CUSTOMER_EXPERIENCE":{"id_column":"PRODUCT_NAME","max_results":10,"search_service":"CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.CUSTOMER_EXPERIENCE_TOOL","title_column":"PRODUCT_NAME"},"DOCUMENTATION_TOOL_SNOW":{"id_column":"SCOPED_FILE_URL","max_results":6,"search_service":"CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.DOCUMENTATION_TOOL_SNOW","title_column":"RELATIVE_PATH"},"SALES_DATA_SEMANTIC_VIEW":{"execution_environment":{"type":"warehouse","warehouse":""},"semantic_view":"CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.SALES_DATA_SEMANTIC_VIEW"}}}
$$;



In [ ]:
GRANT USAGE ON AGENT SNOW_AGENT to role SNOW_ROLE;

# Agent Evaluations

Next step is to validate how the Agent works and define a framework where we can evaluate and compare other Agent versions. 

To acomplish this we are going to use [AI Observability with Snowflake Cortex](https://docs.snowflake.com/en/user-guide/snowflake-cortex/ai-observability)

First step is to create a table where for one question we have a groud truth. We have already a CSV file with some content you can review

In [ ]:
CREATE OR REPLACE TABLE agent_evaluation_data (
    input_query VARCHAR,
    ground_truth OBJECT
);

COPY INTO agent_evaluation_data (
    input_query,
    ground_truth
)
FROM @csv/agent_evaluation_data.csv
FILE_FORMAT = (
    TYPE = 'CSV'
    FIELD_DELIMITER = ','
    SKIP_HEADER = 1
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    NULL_IF = ('NULL', 'null', '')
    ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE
);

In [ ]:
select * from agent_evaluation_data;

You can also programaticaly add some content to the evaluation table:

In [ ]:
INSERT INTO agent_evaluation_data
SELECT
    'What is the total sales for the carvers?',
    OBJECT_CONSTRUCT(
        'tool_name', 'SALES_DATA_SEMANTIC_VIEW',
        'tool_output', OBJECT_CONSTRUCT(
            'SQL', 'WITH __fact_sales AS (
              SELECT
                date_sales,
                article_id,
                customer_id,
                total_price
              FROM CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.FACT_SALES
            ), __dim_article AS (
              SELECT
                article_name,
                article_id
              FROM CC_SNOWFLAKE_INTELLIGENCE_E2E.PUBLIC.DIM_ARTICLE
            )
            SELECT
              SUM(fs.total_price) AS total_sales,
              MIN(fs.date_sales) AS min_date,
              MAX(fs.date_sales) AS max_date,
              COUNT(fs.date_sales) AS transaction_count,
              COUNT(DISTINCT fs.customer_id) AS unique_customers
            FROM __fact_sales AS fs
            INNER JOIN __dim_article AS da
              ON fs.article_id = da.article_id
            WHERE
              da.article_name = ''Carver Skis'' AND fs.date_sales <= CURRENT_DATE
             -- Generated by Cortex Analyst'
        )
    );


In [ ]:
select * from agent_evaluation_data;

In [ ]:
this_user = session.get_current_user()
print (this_user)

In [ ]:
session.sql(f"GRANT IMPERSONATE ON USER {this_user} TO ROLE {this_session_role}").collect()